### 测试vllm本地部署的模型能否使用

In [1]:
! pip install openai

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [6]:
from openai import OpenAI

# Set OpenAI's API key and API base to use vLLM's API server.
api_key = "dnuendocsdbsc-=23i0ern"
base_url = "http://172.16.7.132:23148/v1"  # A800
# base_url = "http://43.136.113.162:31556/v1"

client = OpenAI(base_url=base_url, api_key="ssss")

stream = client.chat.completions.create(
    model="/models/chatglm3-6b",
    messages=[{"role": "user", "content": "什么是大模型，请详细说明，具体能做什么，有哪些方面可以改进，文本长度至少100字"}],
    stream=True,
    max_tokens=1000,
    temperature=0,
)
# 返回迭代ChatCompletionChunk对象
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        if chunk.choices[0].delta.content == '<|im_end|>':
            break
        print(chunk.choices[0].delta.content, end="")


 大模型是指在训练时使用了大量数据和计算资源的深度学习模型。它们通常具有非常高的参数数量和模型复杂度,可以处理大规模的任务,例如语音识别、自然语言处理、计算机视觉等。

大模型可以用于许多不同的领域,例如自然语言处理、计算机视觉、语音识别等。它们可以实现许多任务,例如文本分类、情感分析、机器翻译、图像生成等。

大模型可以做出非常准确的预测,但它们的训练和部署需要大量的计算资源和时间。此外,大模型可能会面临过拟合和模式崩溃等问题,因此需要进行适当的优化和调整。

改进大模型的方法有很多,例如使用正则化技术、早停技术、Dropout等方法来避免过拟合和模式崩溃。此外,还可以通过使用更小的模型、更小的数据集、更快的计算资源等方式来提高模型的效率和性能。

大模型在许多领域都有广泛的应用,但需要进行适当的优化和调整才能发挥出最佳的性能。<|im_end|>

### demo 测试

In [2]:
!pip install json5

  Using cached json5-0.9.25-py3-none-any.whl.metadata (30 kB)
Using cached json5-0.9.25-py3-none-any.whl (30 kB)
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
! pip install -U qwen-agent

  Using cached qwen_agent-0.0.6-py3-none-any.whl.metadata (8.6 kB)
  Using cached dashscope-1.20.1-py3-none-any.whl.metadata (6.6 kB)
  Using cached eval_type_backport-0.2.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached html2text-2024.2.26-py3-none-any.whl
  Using cached jsonlines-4.0.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached python_docx-1.1.2-py3-none-any.whl.metadata (2.0 kB)
  Using cached python_pptx-0.6.23-py3-none-any.whl.metadata (18 kB)
  Using cached rank_bm25-0.2.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached snowballstemmer-2.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached jupyter-1.0.0-py2.py3-none-any.whl.metadata (995 bytes)
  Using cached notebook-7.2.1-py3-none-any.whl.metadata (10 kB)
  Using cached qtconsole-5.5.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached nbconvert-7.16.4-py3-none-any.whl.metadata (8.5 kB)
  Using cached ipywidgets-8.1.3-py3-none-any.whl.metada

In [1]:
import pprint
import urllib.parse
import json5
from qwen_agent.agents import Assistant
from qwen_agent.tools.base import BaseTool, register_tool

In [2]:
import os
os.environ['AMAP_TOKEN'] = '812f1c50ed0cbc0fa79c5ddcf7e2b0ec'

In [8]:
import os
from qwen_agent.agents import Assistant
llm_cfg = {
    # 使用 DashScope 提供的模型服务：
    # 'model': 'qwen-max',
    # 'model_server': 'dashscope',
    # 'api_key': 'YOUR_DASHSCOPE_API_KEY',
    # 如果这里没有设置 'api_key'，它将读取 `DASHSCOPE_API_KEY` 环境变量。

    # 使用与 OpenAI API 兼容的模型服务，例如 vLLM 或 Ollama：
    'model': '/models/chatglm3-6b',
    'model_server': 'http://172.16.7.132:23148/v1',  # base_url，也称为 api_base
    # 'api_key': 'EMPTY',
    # "AMAP_TOKEN":"812f1c50ed0cbc0fa79c5ddcf7e2b0ec"

    # （可选） LLM 的超参数：
    # 'generate_cfg': {
    #     'top_p': 0.8
    # }
}
tools = ['amap_weather']  # image_gen and code_interpreter is a built-in tool in Qwen-Agent

system = '你扮演一个天气预报助手，你具有查询天气的能力。如果不需要查询天气，不用回复'

bot = Assistant(llm=llm_cfg,
                system_message=system,
                function_list=tools,
                )


messages = [{'role': 'user', 'content': '深圳今天的天气怎么样'}]
for response in bot.run(messages=messages):
    print('bot response:', response)

"""Running results：
[
    {'role': 'assistant', 'content': '', 'function_call': {'name': 'my_image_gen', 'arguments': '{"prompt": "a cute cat"}'}},
    {'role': 'function', 'content': '{"image_url": "https://image.pollinations.ai/prompt/a%20cute%20cat"}', 'name': 'my_image_gen'},
    {'role': 'assistant', 'content': "Here is the image of a cute cat based on your description:\n\n![](https://image.pollinations.ai/prompt/a%20cute%20cat)\n\nNow, let's proceed with downloading this image and performing an operation such as flipping it."},
    {'role': 'assistant', 'content': '', 'function_call': {'name': 'code_interpreter', 'arguments': '```py\nfrom PIL import Image\nimport requests\n\n# Downloading the cute cat image\nurl = "https://image.pollinations.ai/prompt/a%20cute%20cat"\nresponse = requests.get(url)\nwith open("cute_cat.jpg", \'wb\') as file:\n    file.write(response.content)\n\n# Flipping the image horizontally\ndef flip_image_horizontally(filename):\n    img = Image.open(filename)\n    flipped_img = img.transpose(Image.FLIP_LEFT_RIGHT)\n    flipped_img.save("flipped_cute_cat.jpg")\n    return flipped_img\n\n# Performing the horizontal flip\nflipped_cat = flip_image_horizontally("cute_cat.jpg")\n```'}},
    {'role': 'function', 'content': 'Finished execution.', 'name': 'code_interpreter'},
    {'role': 'assistant', 'content': 'The image of the cute cat has been downloaded and flipped horizontally. The flipped image has been saved as "flipped_cute_cat.jpg". Since we\'re in a text-based environment, I can\'t display the actual image here, but you can check it out at the location where the script was executed.'}
]
"""

bot response: [{'role': 'assistant', 'content': '您'}]
bot response: [{'role': 'assistant', 'content': '您想'}]
bot response: [{'role': 'assistant', 'content': '您想查询'}]
bot response: [{'role': 'assistant', 'content': '您想查询深圳'}]
bot response: [{'role': 'assistant', 'content': '您想查询深圳今天的'}]
bot response: [{'role': 'assistant', 'content': '您想查询深圳今天的天气'}]
bot response: [{'role': 'assistant', 'content': '您想查询深圳今天的天气，'}]
bot response: [{'role': 'assistant', 'content': '您想查询深圳今天的天气，是'}]
bot response: [{'role': 'assistant', 'content': '您想查询深圳今天的天气，是吗'}]
bot response: [{'role': 'assistant', 'content': '您想查询深圳今天的天气，是吗？'}]
bot response: [{'role': 'assistant', 'content': '您想查询深圳今天的天气，是吗？<'}]
bot response: [{'role': 'assistant', 'content': '您想查询深圳今天的天气，是吗？<|'}]
bot response: [{'role': 'assistant', 'content': '您想查询深圳今天的天气，是吗？<|im'}]
bot response: [{'role': 'assistant', 'content': '您想查询深圳今天的天气，是吗？<|im_'}]
bot response: [{'role': 'assistant', 'content': '您想查询深圳今天的天气，是吗？<|im_end'}]
bot response: [{'role':

'Running results：\n[\n    {\'role\': \'assistant\', \'content\': \'\', \'function_call\': {\'name\': \'my_image_gen\', \'arguments\': \'{"prompt": "a cute cat"}\'}},\n    {\'role\': \'function\', \'content\': \'{"image_url": "https://image.pollinations.ai/prompt/a%20cute%20cat"}\', \'name\': \'my_image_gen\'},\n    {\'role\': \'assistant\', \'content\': "Here is the image of a cute cat based on your description:\n\n![](https://image.pollinations.ai/prompt/a%20cute%20cat)\n\nNow, let\'s proceed with downloading this image and performing an operation such as flipping it."},\n    {\'role\': \'assistant\', \'content\': \'\', \'function_call\': {\'name\': \'code_interpreter\', \'arguments\': \'```py\nfrom PIL import Image\nimport requests\n\n# Downloading the cute cat image\nurl = "https://image.pollinations.ai/prompt/a%20cute%20cat"\nresponse = requests.get(url)\nwith open("cute_cat.jpg", \'wb\') as file:\n    file.write(response.content)\n\n# Flipping the image horizontally\ndef flip_imag